This is my copy of a notebook started by [Yegor Biryukov](https://www.kaggle.com/yegorbiryukov/gfootball-with-memory-patterns) with the deterministic script from [Raffaele Morganti](https://www.kaggle.com/raffaelemorganti/rule-based-way). My sincere gratitude to both of them for making my participation in this competition much easier.

The approach is rule-based and strictly human learning. :) I took the liberty of combining chunks of code from Yegor's notebook into one big chunk for easy execution. Here's the list of enhancements I made.

* Defensive issues:
1. Limit blind Action.Shot up field to goalie only. The reason for this is I found that trying to boot the ball upfield without looking at whether opponent players might be blocking is dangerous so I limit it to goalie who mostly has enough room in front of him to clear.
2. For other players, I try to evade the chasing opponents before trying to pass while avoiding going into my own penalty box.
3. Sliding is essential but doing it wildly can get your player sent off so I chose to slide only when the player is near both the ball and the opponent player for two steps. This is all trial-and-error.
4. High ball overhead beyond our last defender leads to many goals so I handle this situation as a special case. The last defender does not chase the ball but chase the highest opponent player instead. This works reasonably well but not foolproof for the case of two-on-two and the ball is going to the second-highest attacker instead of the highest one. I try to fix it only to make it worse so I gave up.

* Offensive issues:
1. I notice that quick passing and shooting without taking possession of the ball is the way top players score so I mimic it by calculating whether the active player is going to be the first to the ball. If so, issue the pass or shot command before the ball arrives. This is not without a downside in that if the player turns out not to be first to the ball, he can stray too far from the opponent to impede him going for goal. Another sticky point is the fact that passing command is delayed. If the player turns the wrong direction after taking possession of the ball, i.e. facing toward our own goal, it can result in an own goal. So, I made quick passing and shooting sticky by repeating it. This is risky so I try to delay making this quick pass/shoot decision as late as possible before the ball reaches the player.
2. When trying to dribble past defenders, I adjust the constant so we don't look at defenders that is a little bit far behind us. This improves the score quite a bit.

Overall, I'm happy with the result. I think I achieve rank 31st only because I had many submissions with different tuning parameters. What I thought was a good change turn out not to be that good in the end. The randomness of game results results in misleading LB score makes tuning difficult. I wish I have implemented a local LB but it might not help much because of the lack of variety in agents.

# *The "goal" ;) of this notebook is to popularize, to the extent of my modest abilities, usage of functional programming and readable, scalable architectures.*

# Explanation of The Architecture
* In <code><b>agent</b></code> function data of the current environment is modified and <code><b>get_action_of_agent</b></code> function is called to get the action that will be returned.
* In <code><b>get_action_of_agent</b></code> function list of memory patterns is provided by <code><b>find_patterns</b></code> function, then action of the first memory pattern in that list, whose constraints are met by current environment, is returned.
* All Groups of Memory Patterns and all Memory Patterns are just functions that are taking the same arguments: <code><b>obs, player_x, player_y</b></code> and return a dictionary with at least two functions.
* First function in that dictionary for both Groups of Memory Patterns and Memory Patterns is <code><b>environment_fits</b></code>, it returns True or False, in case constraints described in this function are met by current environment or not. If this function returns True, than the Group of Memory Patterns or Memory Pattern it belongs to is selected for future processing. If this function returns False, than search for appropriate Group of Memory Patterns or Memory Pattern continues.
* Second function is called only when this Group of Memory Patterns or Memory Pattern was selected for future processing. In any Group of Memory Patterns the second function is <code><b>get_memory_patterns</b></code> that return list of memory patterns. In any Memory Pattern the second function is <code><b>get_action</b></code> that return action of the memory pattern it belongs to.
* In all Groups of Memory Patterns and all Memory Patterns functions <code><b>environment_fits</b></code>, <code><b>get_memory_patterns</b></code> and <code><b>get_action</b></code> take <code><b>obs, player_x, player_y</b></code> as arguments.

In [ ]:
# Install:
# Kaggle environments.
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.8 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib
!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.8.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
# Custom files
!wget https://gist.githubusercontent.com/RaffaeleMorganti/04192739d0a5a518ac253889eb83c6f1/raw/c09f3d602ea89e66daeda96574d966949a2896ce/11_vs_11_deterministic.py -O football/gfootball/scenarios/11_vs_11_deterministic.py 
!wget https://gist.githubusercontent.com/RaffaeleMorganti/04192739d0a5a518ac253889eb83c6f1/raw/c09f3d602ea89e66daeda96574d966949a2896ce/football_action_set.py -O football/gfootball/env/football_action_set.py
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

In [ ]:
%%writefile submission.py

# start executing cells from here to rewrite submission.py

from kaggle_environments.envs.football.helpers import *
import math
import random

def find_patterns(obs, player_x, player_y):
    """ find list of appropriate patterns in groups of memory patterns """
    for get_group in groups_of_memory_patterns:
        group = get_group(obs, player_x, player_y)
        if group["environment_fits"](obs, player_x, player_y):
            return group["get_memory_patterns"](obs, player_x, player_y)

def get_action_of_agent(obs, player_x, player_y):
    """ get action of appropriate pattern in agent's memory """
    memory_patterns = find_patterns(obs, player_x, player_y)
    # find appropriate pattern in list of memory patterns
    for get_pattern in memory_patterns:
        pattern = get_pattern(obs, player_x, player_y)
        if pattern["environment_fits"](obs, player_x, player_y):
            return pattern["get_action"](obs, player_x, player_y)
        
def get_active_sticky_action(obs, exceptions):
    """ get release action of the first active sticky action, except those in exceptions list """
    release_action = None
    for k in sticky_actions:
        if k not in exceptions and sticky_actions[k] in obs["sticky_actions"]:
            if k == "sprint":
                release_action = Action.ReleaseSprint
            '''
            elif k == "dribble":
                release_action = Action.ReleaseDribble
            else:
                release_action = Action.ReleaseDirection
            '''
            break
    return release_action

def get_active_sticky_direction(obs, exceptions):
    for k in sticky_actions:
        if k not in exceptions and sticky_actions[k] in obs["sticky_actions"]:
            return sticky_actions[k]
    return None

def get_average_distance_to_opponents(obs, player_x, player_y):
    """ get average distance to closest opponents """
    distances_sum = 0
    distances_amount = 0
    for i in range(1, len(obs["right_team"])):
        # if opponent is ahead of player
        if obs["right_team"][i][0] > (player_x - 0.015):
            distance_to_opponent = get_distance(player_x, player_y, obs["right_team"][i][0], obs["right_team"][i][1])
            if distance_to_opponent < 0.1:
                distances_sum += distance_to_opponent
                distances_amount += 1
    # if there is no opponents close around
    if distances_amount == 0:
        return 100, distances_amount
    return distances_sum*10000 // (distances_amount*10), distances_amount

def get_distance(x1, y1, x2, y2):
    """ get two-dimensional Euclidean distance, considering y size of the field """
    return math.sqrt((x1 - x2) ** 2 + (y1 * 2.38 - y2 * 2.38) ** 2)

# Evaluate angle between two objects
def get_angle(pos1,pos2):
    return math.degrees(math.atan2(pos2[1]* 2.38 - pos1[1]* 2.38, pos2[0]-pos1[0]))

# Offense Patterns

def continue_after_gain_poss(obs, player_x, player_y):
    """ continue going the direction that help gain the ball for 3 steps """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # We have the ball but we didn't previously.
        if obs["ball_owned_team"] == 0 and state.prev_ball_owned_team != obs["ball_owned_team"]:
            state.pass_cmd_issued = False # Clear old state
            dir = get_active_sticky_direction(obs, ["sprint","dribble"])
            if dir != None:
                return True
        return False
    
    def get_action(obs, player_x, player_y):
        """ get action of this memory pattern """
        if abs(player_x) > 0.9 and obs.left_team_direction[obs.active][0] < -0.006:
            dir = Action.Top if player_y < 0 else Action.Bottom
            return dir
        if player_x < -0.75:
            # Sensitive area, release sprint first
            if Action.Sprint in obs["sticky_actions"]:
                return Action.ReleaseSprint
            dir = Action.Top if player_y < 0 else Action.Bottom
            return dir
        dir = get_active_sticky_direction(obs, ["sprint","dribble"])
        if state.debug_offense:
            print(f'Step {3001-steps_left} continue_after_gain_poss() {dir} {obs.active}:{player_x:.3f}:{player_y:.3f}')
        if dir != None:
            return do_actions([dir])
        return Action.Shot
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def bad_angle_short_pass(obs, player_x, player_y):
    """ perform a short pass, if player is at bad angle to opponent's goal """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # player have the ball and is at bad angle to opponent's goal
        if (obs["ball_owned_player"] == obs["active"] and
                obs["ball_owned_team"] == 0 and
                abs(player_y) > 0.1 and
                player_x > 0.78):
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        if state.debug_offense:
            print(f'Step {3001-steps_left} bad_angle_short_pass() {obs.active}:{player_x:.3f}:{player_y:.3f}')
        """ get action of this memory pattern """
        action_to_release = get_active_sticky_action(obs, [])
        if action_to_release != None:
            return action_to_release
        if state.pass_cmd_issued == False:
            state.pass_cmd_issued = True
            return Action.ShortPass if abs(player_y) < 0.3 else Action.HighPass
        else:
            if player_x < 0.85:
                action = Action.TopRight if player_y > 0 else Action.BottomRight
            else:
                action = Action.Top if player_y > 0 else Action.Bottom
            return action
    return {"environment_fits": environment_fits, "get_action": get_action}

def close_to_goalkeeper_shot(obs, player_x, player_y):
    """ shot if close to the goalkeeper """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        goalkeeper_x = obs["right_team"][0][0] + obs["right_team_direction"][0][0] * 13
        goalkeeper_y = obs["right_team"][0][1] + obs["right_team_direction"][0][1] * 13
        # player have the ball and located close to the goalkeeper
        if (obs["ball_owned_player"] == obs["active"] and
                obs["ball_owned_team"] == 0 and
                get_distance(player_x, player_y, goalkeeper_x, goalkeeper_y) < 0.3):
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        if state.debug_offense:
            print(f'Step {3001-steps_left} close_to_goalkeeper_shot() {my_dist_opp[obs.active][0]:0.3f} {obs.active}:{player_x:.3f}:{player_y:.3f}')
        """ get action of this memory pattern """
        if Action.Sprint in obs["sticky_actions"]:
            return Action.ReleaseSprint
        if player_y <= -0.03 or (player_y > 0 and player_y < 0.03):
            if Action.BottomRight not in obs["sticky_actions"]:
                return do_actions([Action.BottomRight,Action.Shot])
        else:
            if Action.TopRight not in obs["sticky_actions"]:
                return do_actions([Action.TopRight,Action.Shot])
        return Action.Shot
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def far_from_goal_goalie_has_ball(obs, player_x, player_y):
    """ perform a shot, if far from opponent's goal """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # Goalie have the ball but a defender is active. Spread out and HighPass is fine.
        if (obs["ball_owned_player"] != obs["active"] and
                obs["ball_owned_team"] == 0 and
                (player_x < -0.6 or obs["ball_owned_player"] == 0)):
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        if state.debug_offense:
            print(f'Step {3001-steps_left} far_from_goal_goalie_has_ball() {obs.active}:{player_x:.3f}:{player_y:.3f}')
        """ get action of this memory pattern """
        # The goal here is to get open and not too far from the goalie
        # so he can throw the ball to the active player.
        action = Action.Right
        if player_x > -0.8:
            action = Action.Left
        elif player_x < -0.95:
            action = Action.Right
        elif abs(player_y) < 0.12:
            action = Action.Bottom if player_y > 0 else Action.Top
        return action
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def angle_to_direction(angle):
    if angle > 157.5: return Action.Left, 180
    elif angle > 112.5: return Action.BottomLeft, 135
    elif angle > 67.5: return Action.Bottom, 90
    elif angle > 22.5: return Action.BottomRight, 45
    elif angle > -22.5: return Action.Right, 0
    elif angle > -67.5: return Action.TopRight, -45
    elif angle > -112.5: return Action.Top, -90
    elif angle > -157.5: return Action.TopLeft, -135
    else: return Action.Left, -180

def find_evade_dir(obs, player_x, player_y):
    biggest_distance = 0
    left, left_opponents_amount = get_average_distance_to_opponents(obs, player_x - 0.01, player_y)
    right, right_opponents_amount = get_average_distance_to_opponents(obs, player_x + 0.01, player_y)
    top_right, tr_opponents_amount = get_average_distance_to_opponents(obs, player_x + 0.01, player_y - 0.01)
    top_left, tl_opponents_amount = get_average_distance_to_opponents(obs, player_x - 0.01, player_y - 0.01)
    top, t_opponents_amount = get_average_distance_to_opponents(obs, player_x, player_y - 0.01)
    if player_y <= -0.32:
        top_right = top_left = top = 0
    bottom_right, br_opponents_amount = get_average_distance_to_opponents(obs, player_x + 0.01, player_y + 0.01)
    bottom_left, bl_opponents_amount = get_average_distance_to_opponents(obs, player_x - 0.01, player_y + 0.01)
    bottom, b_opponents_amount = get_average_distance_to_opponents(obs, player_x, player_y + 0.01)
    if player_y >= 0.32:
        bottom_right = bottom_left = bottom = 0
    if player_x <= -0.7 and player_y >= -0.2:
        bottom_left = left = 0
    if player_x <= -0.7 and player_y <= 0.2:
        top_left = left = 0
    if player_x <= -0.85:
        top_left = bottom_left = left = 0
    if player_x > 0.93:
        top_left = top_right = right = 0
    biggest_distance = max(left, right, top_right, top_left, top, bottom_right, bottom_left, bottom)
    action = Action.Right
    if (biggest_distance == right or right == 100):
        # Right is best or clear and already in the center or wing
        action = Action.Right
    elif (biggest_distance == top_right or top_right == 100):
        action = Action.TopRight
    elif (biggest_distance == bottom_right or bottom_right == 100):
        action = Action.BottomRight
    elif (biggest_distance == top or top == 100):
        action = Action.Top
    elif (biggest_distance == bottom or bottom == 100):
        action = Action.Bottom
    elif (biggest_distance == top_left or top_left == 100):
        action = Action.TopLeft
    elif (biggest_distance == bottom_left or bottom_left == 100):
        action = Action.BottomLeft
    elif (biggest_distance == left or left == 100):
        action = Action.Left
    return action

def far_from_goal_shot(obs, player_x, player_y):
    """ perform a shot, if far from opponent's goal """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # player have the ball and is far from opponent's goal
        if (obs["ball_owned_player"] == obs["active"] and
                obs["ball_owned_team"] == 0 and
                obs["ball_owned_player"] == 0):
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        if state.debug_offense:
            print(f'Step {3001-steps_left} far_from_goal_shot() {obs.active}:{player_x:.3f}:{player_y:.3f}')
        """ get action of this memory pattern """
        return Action.Shot
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def far_from_goal_high_pass(obs, player_x, player_y):
    """ perform a high pass, if far from opponent's goal """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # player have the ball and is far from opponent's goal
        if (obs["ball_owned_player"] == obs["active"] and
            obs["ball_owned_team"] == 0 and
            player_x < -0.3):
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        """ get action of this memory pattern """
        if state.debug_offense:
            print(f'Step {3001-steps_left} far_from_goal_high_pass() {obs.active}:{player_x:.3f}:{player_y:.3f}')
        if defender_close[obs.active] == 0 or abs(player_y) > 0.33:
            should_pass, ix = teammate_open(obs, player_x, player_y)
            if should_pass:
                pass_angle = my_angle_own[obs.active][ix]
                direction, angle = angle_to_direction(pass_angle)
                action = Action.ShortPass if defender_block_pass[obs.active][ix] == 0 else Action.HighPass
            else:
                direction = Action.Right
                action = Action.HighPass
            if state.pass_cmd_issued == False:
                state.pass_cmd_issued = True
            if direction not in obs["sticky_actions"]:
                return do_actions([direction,action])
            else:
                return action
        else:
            direction = find_evade_dir(obs, player_x, player_y)
            curdir = get_active_sticky_direction(obs, ["sprint","dribble"])
            if curdir != None and curdir != direction:
                if Action.Sprint in obs["sticky_actions"]:
                    return Action.ReleaseSprint
            return direction
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def go_through_opponents(obs, player_x, player_y):
    """ avoid closest opponents by going around them """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        biggest_distance = 0
        right, right_opponents_amount = get_average_distance_to_opponents(obs, player_x + 0.01, player_y)
        top_right, tr_opponents_amount = get_average_distance_to_opponents(obs, player_x + 0.01, player_y - 0.01)
        top, t_opponents_amount = get_average_distance_to_opponents(obs, player_x, player_y - 0.01)
        if player_y <= -0.3:
            top_right = top = 0
        bottom_right, br_opponents_amount = get_average_distance_to_opponents(obs, player_x + 0.01, player_y + 0.01)
        bottom, b_opponents_amount = get_average_distance_to_opponents(obs, player_x, player_y + 0.01)
        if player_y >= 0.3:
            bottom_right = bottom = 0
        biggest_distance = max(right, top_right, bottom_right)
        biggest_opponents_amount = max(right_opponents_amount,tr_opponents_amount,br_opponents_amount)
        action = Action.ReleaseDribble
        if right < 100 and top_right < 100 and bottom_right < 100:
            # then biggest_distance wins
            if biggest_distance == right:
                # Right is best or clear
                action = Action.Right
            elif biggest_distance == top_right and player_y > -0.32:
                action = Action.TopRight
            elif biggest_distance == bottom_right and player_y < 0.32:
                action = Action.BottomRight
        elif right == 100 and top_right < 100 and bottom_right < 100:
            action = Action.Right
        elif right < 100 and top_right == 100 and bottom_right < 100:
            action = Action.TopRight
        elif right < 100 and top_right < 100 and bottom_right == 100:
            action = Action.BottomRight
        elif right == 100 and top_right == 100 and bottom_right < 100:
            action = Action.Right if player_y < 0.12 else Action.TopRight
        elif right == 100 and top_right < 100 and bottom_right == 100:
            action = Action.Right if player_y > -0.12 else Action.BottomRight
        elif right < 100 and top_right == 100 and bottom_right == 100:
            action = Action.TopRight if player_y > 0 else Action.BottomRight
        elif right == 100 and top_right == 100 and bottom_right == 100:
            if abs(player_y) < 0.12:
                action = Action.Right
            elif player_y > 0.12:
                action = Action.TopRight
            else:
                action = Action.BottomRight
            
        obs["memory_patterns"]["go_around_opponent"] = action
        obs["memory_patterns"]["biggest_distance"] = biggest_distance
        if state.debug_offense:
            print(f'  go_through ({player_x:.3f},{player_y:.3f}) {action} R:TR:BR {right:.3f}:{top_right:.3f}:{bottom_right:.3f}'
                  f' {right_opponents_amount}:{tr_opponents_amount}:{br_opponents_amount}')
        # is player is surrounded?
        if biggest_opponents_amount >= 3:
            obs["memory_patterns"]["go_around_opponent_surrounded"] = True
        else:
            obs["memory_patterns"]["go_around_opponent_surrounded"] = False
        return True
        
    def get_action(obs, player_x, player_y):
        """ get action of this memory pattern """
        action = obs["memory_patterns"]["go_around_opponent"]
        if state.debug_offense:
            print(f'Step {3001-steps_left} go_through_opponents() {action} {obs.active}:{player_x:.3f}:{player_y:.3f}')
        if (obs["memory_patterns"]["go_around_opponent_surrounded"] or action == Action.ReleaseDribble):
            # if player is surrounded or no good dir to go, stop.
            if Action.Sprint in obs["sticky_actions"]:
                return Action.ReleaseSprint
            return Action.ReleaseDribble
        if Action.Sprint not in obs["sticky_actions"]:
            return Action.Sprint
        return action
    
    return {"environment_fits": environment_fits, "get_action": get_action}

# Defense Patterns

def teammate_open(obs, player_x, player_y):
    active = obs["active"]
    active_potential = potential[active]
    sort_potential = sorted(potential.items(), key=lambda item: item[1], reverse=True)
    sorted_opp_pos = sorted(obs['right_team'])
    current_offside_x_value = sorted_opp_pos[-2][0]
    for pot in sort_potential[0:1]:
        ix = pot[0]
        if ix == active or ix == 0:
            # Don't send to ourselves or goalie
            continue
        if obs['left_team_active'][ix] is False:
            continue
        if pot[1] <= active_potential+0.01:
            if (state.debug_passing):
                print(f'Step {3001-steps_left} Ignore {ix} pot:{pot[1]:.3f} our:{active_potential:.3f}')
            continue
        if (obs['left_team'][ix][0] > 0 and obs['left_team'][ix][0] > current_offside_x_value and
            obs['left_team'][active][0] <= current_offside_x_value):
            if state.debug_offense:
                print(f'Step {3001-steps_left} teammate_open Act:{obs.active}X:{player_x:.3f}'
                      f' ignore {ix} mateX:{obs.left_team[ix][0]:.3f} offsideX:{current_offside_x_value:.3f}')
            continue
        if obs['left_team'][ix][0] > 0.88 and abs(obs['left_team'][ix][1]) > 0.28:
            # Ignore those that are too deep and too outside
            continue
        if state.debug_offense:
            print(f'Step {3001-steps_left} teammate_open Act:{obs.active}:{player_x:.3f}:{player_y:.3f}'
                  f' pass to mate {ix}:{obs.left_team[ix][0]:.3f}:{obs.left_team[ix][1]:.3f} pot:{pot[1]} vs {active_potential}')
        return True, ix
    return False, 0

def charge_and_shot(obs, player_x, player_y):
    """ when player's center and the cross is coming in, shoot first time """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        if obs["ball_owned_team"] != -1:
            return False
        if player_x > 0.7 and player_x < 1.01 and abs(player_y) < 0.12:
            # offensive player in the box
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        """ get action of this memory pattern """
        shot_angle = my_angle_goal[obs.active]
        direction, angle = angle_to_direction(shot_angle)
        if state.debug_offense:
            print(f'Step {3001-steps_left} charge_and_shot() {direction} A:{shot_angle} {obs.active}:{player_x:.3f}:{player_y:.3f}')
        return do_actions([direction,Action.Shot])
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def quick_pass(obs, player_x, player_y):
    """ when player's center and the cross is coming in, shoot first time """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        if obs["ball_owned_team"] != -1:
            return False
        if abs(obs["ball_direction"][0])+abs(obs["ball_direction"][1]) < 0.007:
            # Ball moving slowly, it's most likely not a passing situation
            return False
        if obs["active"] == 0:
            # Have goalie clear it first time
            return True
        dist_ball = get_distance(player_x, player_y, obs["ball"][0], obs["ball"][1])
        if state.debug_offense:
            print(f'Step {3001-steps_left} DistB {dist_ball:.3f} {player_x:.3f},{player_y:.3f}'
                  f' d:{obs["left_team_direction"][obs.active][0]:.3f},{obs["left_team_direction"][obs.active][1]:.3f}'
                  f' B:{obs["ball"][0]:.3f},{obs["ball"][1]:.3f} Bd:{obs["ball_direction"][0]:.3f},{obs["ball_direction"][1]:.3f}')
        if dist_ball > 0.15:
            fu_pos, t_pos = ball_intercept_pos(obs)
            if t_pos > 4:
                # Wait until the ball is close before making decision
                return False
        if player_x < 0.4 and player_x > -0.2 and defender_close[obs.active] > 0:
            return False
        if player_x <= -0.2 and defender_near[obs.active] > 0:
            return False
        if player_x > -0.7:
            # Pass quickly if we are not too deep
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        """ get action of this memory pattern """
        if obs["active"] == 0:
            # Have goalie clear it first time
            return Action.Shot
        first, opp = first_to_the_ball(obs, player_x, player_y)
        should_pass, ix = teammate_open(obs, player_x, player_y)
        if first:
            if should_pass is False:
                if state.debug_offense:
                    print(f'Step {3001-steps_left} quick_pass {ix} no open teammate from {obs.active}:{player_x:.3f}:{player_y:.3f}')
                return Action.Idle
            pass_angle = my_angle_own[obs.active][ix]
            direction, angle = angle_to_direction(pass_angle)
            action = Action.ShortPass if defender_block_pass[obs.active][ix] == 0 else Action.HighPass
            if state.debug_offense:
                print(f'Step {3001-steps_left} quick_pass to {ix} A:{pass_angle:.2f} {direction} {action} from {obs.active}:{player_x:.3f}:{player_y:.3f}')
            return do_actions([direction,action])
        else:
            # We won't be first to the ball, just idle to continue what we are doing
            return Action.Idle

    return {"environment_fits": environment_fits, "get_action": get_action}

def steps_to_go_there(loca, va, locb, vb):
    dvx = vb[0]-va[0]
    dvy = vb[1]-va[1]
    tx = ty = 0
    if dvx != 0:
        tx = (loca[0]-locb[0])/dvx
    if dvy != 0:
        ty = (loca[1]-locb[1])/dvy
    if dvx == 0 and dvy == 0:
        return (locb[0],locb[1]), 50
    time = max(abs(tx),abs(ty))
    ex = locb[0]+vb[0]*time
    ey = locb[1]+vb[1]*time
    if state.debug_offense:
        print(f'steps_to_go_there: {loca[0]:.3f},{loca[1]:.3f} to {locb[0]:.3f},{locb[1]:.3f}'
              f' Time {time:.2f} x:{tx:.2f} t:{ty:.2f} loc {ex:.3f},{ey:.3f}')
    return (ex,ey), time
    
# Return True if we are the first to reach the ball,
# compared to other players opp or ours.
def first_to_the_ball(obs, player_x, player_y):
    active = obs['active']
    if obs['ball'][2] > 0.3: # Ball in the air
        # Location that the ball will land and in how many steps
        tgt, time = ball_landing_pos(obs)
        tgtdir = (0,0)
    else:
        tgt = obs['ball']
        tgtdir = obs['ball_direction']

    our_dist = get_distance(player_x, player_y, tgt[0], tgt[1])
    if state.debug_offense:
        print(f'Step {3001-steps_left} first_to_the_ball D:{our_dist:.3f} {player_x:.3f},{player_y:.3f}'
              f' d:{obs["left_team_direction"][obs.active][0]:.3f},{obs["left_team_direction"][obs.active][1]:.3f}'
              f' B:{obs["ball"][0]:.3f},{obs["ball"][1]:.3f} Bd:{obs["ball_direction"][0]:.3f},{obs["ball_direction"][1]:.3f}')
    if our_dist > 0.15:
        maxopp = None
        maxopptime = 50
        # Calculate how long it takes us to get to the target
        loc, ourtime = steps_to_go_there(obs['left_team'][active], obs['left_team_direction'][active],
                                         tgt, tgtdir)
        for ix in range(11):
            if my_dist_opp[active][ix] > 0.4:
                continue
            loc, opptime = steps_to_go_there(obs['right_team'][ix], obs['right_team_direction'][ix],
                                             tgt, tgtdir)
            if opptime < maxopptime:
                maxopptime = opptime
                maxopp = ix
        if maxopp != None and maxopptime < ourtime:
            return False, maxopp
        return True, None
    else:
        maxopp = None
        maxdist = 50
        # We are very close, use distance
        for ix in range(11):
            if my_dist_opp[active][ix] > 0.4:
                continue
            opp_dist = get_distance(obs['right_team'][ix][0], obs['right_team'][ix][1], tgt[0], tgt[1])
            if opp_dist < maxdist:
                maxdist = opp_dist
                maxopp = ix
        if maxopp != None and maxdist < our_dist:
            return False, maxopp
        return True, None
        
def ball_intercept_pos(obs):
    active = obs['active']
    va = obs['left_team_direction'][active]
    loca = obs['left_team'][active]
    vb = obs['ball_direction']
    locb = obs['ball']
    dvx = vb[0]-va[0]
    dvy = vb[1]-va[1]
    tx = ty = 0
    if dvx != 0:
        tx = (loca[0]-locb[0])/dvx
    if dvy != 0:
        ty = (loca[1]-locb[1])/dvy
    if dvx == 0 and dvy == 0:
        return (locb[0],locb[1]), 10
    time = min(abs(tx),abs(ty))
    ex = locb[0]+vb[0]*time
    ey = locb[1]+vb[1]*time
    if state.debug_offense:
        print(f'Ball_intercept_pos: Time {time:.2f} x:{tx:.2f} t:{ty:.2f} loc {ex:.3f},{ey:.3f}')
    return (ex,ey), time

# Estimate landing position
def ball_landing_pos(obs):
    start_height = obs['ball'][2]
    end_height = 0.2
    start_speed = obs['ball_direction'][2]
    gravity = 0.1
    time = (start_speed**2/gravity**2 - 2/gravity*(end_height-start_height))**0.5 + start_speed/gravity
    ex = obs['ball'][0]+obs['ball_direction'][0]*time
    ey = obs['ball'][1]+obs['ball_direction'][1]*time
    if state.debug_offense:
        print(f'ball_landing_pos: Time {time:.2f} loc {ex:.3f},{ey:.3f}')
    return (ex,ey), time

def chase_active_opp(obs, player_x, player_y):
    # if opponent is ahead of player
    opp_active_x = obs["right_team"][obs["ball_owned_player"]][0]
    opp_active_y = obs["right_team"][obs["ball_owned_player"]][1]
    if player_x < -0.69 and abs(player_y) < 0.23:
        # Penalty box, be more careful
        slide_distance = 0.015
    else:
        slide_distance = 0.025
    if opp_active_x < player_x:
        distance_to_opponent = get_distance(player_x, player_y, opp_active_x, opp_active_y)
        distance_to_ball = get_distance(player_x, player_y, obs["ball"][0], obs["ball"][1])
        if (distance_to_opponent < slide_distance and distance_to_ball < slide_distance):
            return True
    state.chase = False
    state.chase_cnt = 0
    return False

def chase_slide(obs, player_x, player_y):
    """ when we are chasing and behind an opp player and both are going full speed """
    """ Count how long we've chased him. On the count of five, slide. """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        chasing = chase_active_opp(obs, player_x, player_y)
        if obs["ball_owned_team"] == 1:
            if chasing == False:
                return False
            state.chase = True
            state.chase_cnt += 1
            if state.debug_defense:
                print(f'Step {3001-steps_left} cnt:{state.chase_cnt} {obs.active}:{player_x:.3f}:{player_y:.3f}')
            if state.chase_cnt >= 2:
                return True
        return False
        
    def get_action(obs, player_x, player_y):
        if state.debug_defense:
            print(f'Step {3001-steps_left} chase_slide() {obs.active}:{player_x:.3f}:{player_y:.3f}')
        """ get action of this memory pattern """
        return Action.Slide
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def mark(obs,tgt,tgt_dir):
    active = obs['active']
    our_dir = obs['left_team_direction'][active]
    our_loc = obs['left_team'][active]
    direction = Action.ReleaseDirection
    if tgt[0] > our_loc[0]+(0.044)-tgt_dir[0]:
        if tgt[1] > our_loc[1]+0.015:
            # Below us and far
            direction = Action.BottomRight
        elif tgt[1] > our_loc[1]:
            direction = Action.Right
        elif tgt[1] < our_loc[1]-0.015:
            direction = Action.TopRight
        else:
            direction = Action.Right
    elif tgt[0] > our_loc[0]+(0.044):
        if tgt[1] > our_loc[1]+0.015:
            # Below us and far
            direction = Action.Bottom
        elif tgt[1] > our_loc[1]:
            direction = Action.Idle
        elif tgt[1] < our_loc[1]-0.015:
            direction = Action.Top
        else:
            direction = Action.Idle
    elif tgt[0] > our_loc[0]:
        # Slightly further from our goal than us on X
        if tgt[1] > our_loc[1]+0.015:
            # Below us and far
            direction = Action.Bottom
        elif tgt[1] > our_loc[1]:
            # Below us, not far
            direction = Action.Left
        elif tgt[1] < our_loc[1]-0.015:
            # Above us, and far
            direction = Action.Top
        else:
            # Above us, not far
            direction = Action.Left
    elif tgt[0] < our_loc[0]:
        # Closer to our goal than us on X
        if tgt[1] > our_loc[1]+0.015:
            # Below us and far
            direction = Action.BottomLeft
        elif tgt[1] > our_loc[1]:
            # Below us, not far
            direction = Action.Left
        elif tgt[1] < our_loc[1]-0.015:
            # Above us and far
            direction = Action.TopLeft
        else:
            # Above us, not far
            direction = Action.Left
    if (state.debug_defense):
        print(f'Step {3001-steps_left} {active} go {direction}, tgt:{tgt[0]:.3f},{tgt[1]:.3f}:{tgt_dir[0]:.3f},{tgt_dir[1]:.3f}'
              f' us:{our_loc[0]:.3f},{our_loc[1]:.3f}:{our_dir[0]:.3f},{our_dir[1]:.3f}') 
    return direction

def find_highest_opp(obs):
    highest_x = 2
    highest = 9
    for ix in range(11):
        if highest_x > obs['right_team'][ix][0]:
            highest_x = obs['right_team'][ix][0]
            highest = ix
    return highest

def high_ball_overhead(obs, player_x, player_y):
    """ High ball overhead, be behind the forward. Don't chase the ball. """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        if obs["ball_owned_team"] != -1:
            return False
        if abs(obs["ball_direction"][0])+abs(obs["ball_direction"][1]) < 0.01:
            # Ball moving slowly, it's most likely not a passing situation
            return False
        # Active is the last defender
        sorted_our_pos = sorted(obs['left_team'])
        last_defender_x_value = sorted_our_pos[1][0]
        if last_defender_x_value >= player_x and obs["ball"][0] > player_x:
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        """ get action of this memory pattern """
        # Find the highest forward and mark him
        highest_opp = find_highest_opp(obs)
        action = mark(obs,obs["right_team"][highest_opp],obs["right_team_direction"][highest_opp])
        if state.debug_defense:
            print(f'Step {3001-steps_left} high_ball_overhead() {action} {obs.active}:{player_x:.3f}:{player_y:.3f}'
                  f' mark {highest_opp}:{obs["right_team"][highest_opp][0]}:{obs["right_team"][highest_opp][1]}'
                  f' B:{obs.ball[0]:.3f}:{obs.ball[1]:.3f}'
                  f' Bd:{obs.ball_direction[0]:.3f}:{obs.ball_direction[1]:.3f}')
        return action
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def run_to_ball_bottom(obs, player_x, player_y):
    """ run to the ball if it is to the bottom from player's position """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # ball is to the bottom from player's position
        if (obs["ball"][1] > player_y and
                abs(obs["ball"][0] - player_x) < 0.01):
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        if state.debug_defense:
            print(f'Step {3001-steps_left} run_to_ball_bottom() {obs.active}:{player_x:.3f}:{player_y:.3f}'
                  f' B:{obs.ball[0]:.3f}:{obs.ball[1]:.3f} Bd:{obs.ball_direction[0]:.3f}:{obs.ball_direction[1]:.3f}')
        """ get action of this memory pattern """
        if Action.Sprint not in obs["sticky_actions"]:
            return Action.Sprint
        return Action.Bottom
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def run_to_ball_bottom_left(obs, player_x, player_y):
    """ run to the ball if it is to the bottom left from player's position """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # ball is to the bottom left from player's position
        if (obs["ball"][0] < player_x and
                obs["ball"][1] > player_y):
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        if state.debug_defense:
            print(f'Step {3001-steps_left} run_to_ball_bottom_left() {obs.active}:{player_x:.3f}:{player_y:.3f}'
                  f' B:{obs.ball[0]:.3f}:{obs.ball[1]:.3f} Bd:{obs.ball_direction[0]:.3f}:{obs.ball_direction[1]:.3f}')
        """ get action of this memory pattern """
        if Action.Sprint not in obs["sticky_actions"]:
            return Action.Sprint
        return Action.BottomLeft
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def run_to_ball_bottom_right(obs, player_x, player_y):
    """ run to the ball if it is to the bottom right from player's position """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # ball is to the bottom right from player's position
        if (obs["ball"][0] > player_x+0.015 and
                obs["ball"][1] > player_y):
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        if state.debug_defense:
            print(f'Step {3001-steps_left} run_to_ball_bottom_right() {obs.active}:{player_x:.3f}:{player_y:.3f}'
                  f' B:{obs.ball[0]:.3f}:{obs.ball[1]:.3f} Bd:{obs.ball_direction[0]:.3f}:{obs.ball_direction[1]:.3f}')
        """ get action of this memory pattern """
        if Action.Sprint not in obs["sticky_actions"]:
            return Action.Sprint
        return Action.BottomRight
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def run_to_ball_left(obs, player_x, player_y):
    """ run to the ball if it is to the left from player's position """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # ball is to the left from player's position
        if (obs["ball"][0] < player_x and
                abs(obs["ball"][1] - player_y) < 0.01):
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        if state.debug_defense:
            print(f'Step {3001-steps_left} run_to_ball_left() {obs.active}:{player_x:.3f}:{player_y:.3f}'
                  f' B:{obs.ball[0]:.3f}:{obs.ball[1]:.3f} Bd:{obs.ball_direction[0]:.3f}:{obs.ball_direction[1]:.3f}')
        """ get action of this memory pattern """
        if Action.Sprint not in obs["sticky_actions"]:
            return Action.Sprint
        return Action.Left
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def run_to_ball_right(obs, player_x, player_y):
    """ run to the ball if it is to the right from player's position """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # ball is to the right from player's position
        if (obs["ball"][0] > player_x+0.04 and
                abs(obs["ball"][1] - player_y) < 0.01):
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        if state.debug_defense:
            print(f'Step {3001-steps_left} run_to_ball_right() {obs.active}:{player_x:.3f}:{player_y:.3f}'
                  f' B:{obs.ball[0]:.3f}:{obs.ball[1]:.3f} Bd:{obs.ball_direction[0]:.3f}:{obs.ball_direction[1]:.3f}')
        """ get action of this memory pattern """
        if Action.Sprint not in obs["sticky_actions"]:
            return Action.Sprint
        return Action.Right
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def run_to_ball_top(obs, player_x, player_y):
    """ run to the ball if it is to the top from player's position """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # ball is to the top from player's position
        if (obs["ball"][1] < player_y and
                abs(obs["ball"][0] - player_x) < 0.01):
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        if state.debug_defense:
            print(f'Step {3001-steps_left} run_to_ball_top() {obs.active}:{player_x:.3f}:{player_y:.3f}'
                  f' B:{obs.ball[0]:.3f}:{obs.ball[1]:.3f} Bd:{obs.ball_direction[0]:.3f}:{obs.ball_direction[1]:.3f}')
        """ get action of this memory pattern """
        if Action.Sprint not in obs["sticky_actions"]:
            return Action.Sprint
        return Action.Top
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def run_to_ball_top_left(obs, player_x, player_y):
    """ run to the ball if it is to the top left from player's position """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # ball is to the top left from player's position
        if (obs["ball"][0] < player_x and
                obs["ball"][1] < player_y):
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        if state.debug_defense:
            print(f'Step {3001-steps_left} run_to_ball_top_left() {obs.active}:{player_x:.3f}:{player_y:.3f}'
                  f' B:{obs.ball[0]:.3f}:{obs.ball[1]:.3f} Bd:{obs.ball_direction[0]:.3f}:{obs.ball_direction[1]:.3f}')
        """ get action of this memory pattern """
        if Action.Sprint not in obs["sticky_actions"]:
            return Action.Sprint
        return Action.TopLeft
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def run_to_ball_top_right(obs, player_x, player_y):
    """ run to the ball if it is to the top right from player's position """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # ball is to the top right from player's position
        if (obs["ball"][0] > player_x+0.015 and
                obs["ball"][1] < player_y):
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        if state.debug_defense:
            print(f'Step {3001-steps_left} run_to_ball_top_right() {obs.active}:{player_x:.3f}:{player_y:.3f}'
                  f' B:{obs.ball[0]:.3f}:{obs.ball[1]:.3f} Bd:{obs.ball_direction[0]:.3f}:{obs.ball_direction[1]:.3f}')
        """ get action of this memory pattern """
        if Action.Sprint not in obs["sticky_actions"]:
            return Action.Sprint
        return Action.TopRight
    
    return {"environment_fits": environment_fits, "get_action": get_action}

# Special Game Modes
# Evaluate euclidian distance between two objects
def uncalibrated_get_distance(pos1,pos2):
    return ((pos1[0]-pos2[0])**2+(pos1[1]-pos2[1])**2)**0.5

def defending_setpiece(obs):
    # Find out if we are the kicking team or the defending team
    ball_dist = uncalibrated_get_distance(obs["left_team"][obs['active']], obs['ball'])
    defending = True
    if (ball_dist < 0.043 or 
        (obs['game_mode'] == GameMode.Penalty and ball_dist < 0.056)):
         # ball is that far for goalkick, kickoff is 0.02, pen is 0.055
        defending = False
    return defending

def go_center(player_x, player_y):
    action = Action.Top if player_y > 0 else Action.Bottom
    if player_x > -0.7:
        action = Action.Left
    elif player_x < -0.9:
        action = Action.Right
    return action

def corner(obs, player_x, player_y):
    """ perform a high pass in corner game mode """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # it is corner game mode
        if obs['game_mode'] == GameMode.Corner:
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        defend = defending_setpiece(obs)
        if state.debug_special:
            print(f'Step {3001-steps_left} corner() {obs.active}:{player_x:.3f}:{player_y:.3f} {"D" if defend else "A"}')
        """ get action of this memory pattern """
        if defend:
            action = go_center(player_x, player_y)
            return action
        action = Action.Top if player_y > 0 else Action.Bottom
        followthru_action = Action.TopRight if player_y > 0 else Action.BottomRight
        return do_actions([action,Action.HighPass,followthru_action,followthru_action,Action.HighPass,Action.HighPass,Action.HighPass])
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def free_kick(obs, player_x, player_y):
    """ perform a high pass or a shot in free kick game mode """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # it is free kick game mode
        if obs['game_mode'] == GameMode.FreeKick:
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        defend = defending_setpiece(obs)
        if state.debug_special:
            print(f'Step {3001-steps_left} free_kick() {obs.active}:{player_x:.3f}:{player_y:.3f} {"D" if defend else "A"}')
        """ get action of this memory pattern """
        if obs["ball_owned_team"] == 0:
            # shot if player close to goal
            if player_x > 0.5:
                action_to_release = get_active_sticky_action(obs, ["top_right", "bottom_right"])
                if action_to_release != None:
                    return action_to_release
                if Action.TopRight not in obs["sticky_actions"] and Action.BottomRight not in obs["sticky_actions"]:
                    if player_y > 0:
                        return Action.TopRight
                    else:
                        return Action.BottomRight
                return Action.Shot
            else:
                # high pass if player far from goal
                action_to_release = get_active_sticky_action(obs, ["right"])
                if action_to_release != None:
                    return action_to_release
                if Action.Right not in obs["sticky_actions"]:
                    return Action.Right
                return Action.HighPass
        else:
            # We are the defending team
            if defender_close[obs.active] > 0:
                return Action.ReleaseDirection
            if player_x < 0.5:
                # Mark the one behind
                return Action.Right
            else:
                # Mark the one ahead
                return Action.Left
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def goal_kick(obs, player_x, player_y):
    """ perform a short pass in goal kick game mode """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # it is goal kick game mode
        if obs['game_mode'] == GameMode.GoalKick:
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        defend = defending_setpiece(obs)
        if state.debug_special:
            print(f'Step {3001-steps_left} goal_kick() {obs.active}:{player_x:.3f}:{player_y:.3f} {"D" if defend else "A"}')
        """ get action of this memory pattern """
        action_to_release = get_active_sticky_action(obs, ["top_right", "bottom_right"])
        if action_to_release != None:
            return action_to_release
        return Action.TopRight if obs.ball[1] > 0 else Action.BottomRight
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def kick_off(obs, player_x, player_y):
    """ perform a short pass in kick off game mode """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # it is kick off game mode
        defend = defending_setpiece(obs)
        if obs['game_mode'] == GameMode.KickOff:
            return True
        if (obs['active'] != 6 and obs['ball'][0] == 0 and obs['ball'][1] == 0 and
            obs['ball_direction'][0] == 0 and obs['ball_direction'][1] == 0 and
            obs['ball_rotation'][0] == 0 and obs['ball_rotation'][1] == 0):
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        defend = defending_setpiece(obs)
        """ get action of this memory pattern """
        if state.debug_special:
            print(f'Step {3001-steps_left} kick_off() {obs.game_mode}'
                  f' {obs.active}:{player_x:.3f}:{player_y:.3f} {"D" if defend else "A"}'
                  f' B:{obs.ball[0]},{obs.ball[1]}')
        if defend:
            return do_actions([Action.Right,Action.Right,Action.Right,Action.Right])
        else:
            return do_actions([Action.TopLeft,Action.Sprint,Action.Sprint])

    return {"environment_fits": environment_fits, "get_action": get_action}

def penalty(obs, player_x, player_y):
    """ perform a shot in penalty game mode """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # it is penalty game mode
        if obs['game_mode'] == GameMode.Penalty:
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        defend = defending_setpiece(obs)
        if state.debug_special:
            print(f'Step {3001-steps_left} penalty() {obs.active}:{player_x:.3f}:{player_y:.3f} {"D" if defend else "A"}')
        """ get action of this memory pattern """
        return do_actions([Action.ReleaseDribble,Action.Shot,Action.ReleaseSprint,Action.Shot,Action.Bottom,Action.HighPass,Action.ReleaseSprint,Action.Slide,Action.Slide])
    
    return {"environment_fits": environment_fits, "get_action": get_action}

def throw_in(obs, player_x, player_y):
    """ perform a short pass in throw in game mode """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # it is throw in game mode
        if obs['game_mode'] == GameMode.ThrowIn:
            return True
        return False
        
    def get_action(obs, player_x, player_y):
        defend = defending_setpiece(obs)
        if state.debug_special:
            print(f'Step {3001-steps_left} throw_in() {obs.active}:{player_x:.3f}:{player_y:.3f} {"D" if defend else "A"}')
        """ get action of this memory pattern """
        action_to_release = get_active_sticky_action(obs, ["right"])
        if action_to_release != None:
            return action_to_release
        if obs["ball_owned_team"] == 0:
            # We are the throwing team
            if player_y < 0:
                return Action.TopRight
            else:
                return Action.BottomRight
        else:
            # We are the defending team
            if defender_close[obs.active] > 0:
                return Action.ReleaseDirection
            if player_x < 0.5:
                # Mark the one behind
                return Action.Right
            else:
                # Mark the one ahead
                return Action.Left
    
    return {"environment_fits": environment_fits, "get_action": get_action}

# Any environment

def idle(obs, player_x, player_y):
    """ do nothing, release all sticky actions """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        return True
        
    def get_action(obs, player_x, player_y):
        """ get action of this memory pattern """
        action_to_release = get_active_sticky_action(obs, [])
        if action_to_release != None:
            return action_to_release
        return Action.Idle
    
    return {"environment_fits": environment_fits, "get_action": get_action}

# Group of Memory Patterns

def defence_memory_patterns(obs, player_x, player_y):
    """ group of memory patterns for environments in which opponent's team has the ball """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # player don't have the ball
        if obs["ball_owned_team"] != 0:
            return True
        return False
        
    def get_memory_patterns(obs, player_x, player_y):
        """ get list of memory patterns """
        # shift ball position
        obs["ball"][0] += obs["ball_direction"][0] * 7
        obs["ball"][1] += obs["ball_direction"][1] * 3
        # if opponent has the ball and is far from y axis center
        if abs(obs["ball"][1]) > 0.07 and obs["ball_owned_team"] == 1:
            obs["ball"][0] -= 0.01
            if obs["ball"][1] > 0:
                obs["ball"][1] -= 0.01
            else:
                obs["ball"][1] += 0.01
            
        memory_patterns = [
            charge_and_shot,
            quick_pass,
            run_to_ball_right,
            run_to_ball_left,
            run_to_ball_bottom,
            run_to_ball_top,
            run_to_ball_top_right,
            run_to_ball_top_left,
            run_to_ball_bottom_right,
            run_to_ball_bottom_left,
            idle
        ]
        return memory_patterns
        
    return {"environment_fits": environment_fits, "get_memory_patterns": get_memory_patterns}

def regular_defence_memory_patterns(obs, player_x, player_y):
    """ group of memory patterns for environments in which opponent's team has the ball """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # We are defending
        chasing = chase_active_opp(obs, player_x, player_y)
        if obs["ball_owned_team"] == 1 and chasing:
            return True
        # Active is the last defender
        sorted_our_pos = sorted(obs['left_team'])
        last_defender_x_value = sorted_our_pos[1][0]
        if (obs["ball_owned_team"] == -1 and
            abs(obs["ball_direction"][0])+abs(obs["ball_direction"][1]) > 0.01 and
            last_defender_x_value >= player_x and obs["ball"][0] > player_x):
            return True
        return False
        
    def get_memory_patterns(obs, player_x, player_y):
        """ get list of memory patterns """
        memory_patterns = [
            chase_slide,
            high_ball_overhead,
            idle
        ]
        return memory_patterns
        
    return {"environment_fits": environment_fits, "get_memory_patterns": get_memory_patterns}

def goalkeeper_memory_patterns(obs, player_x, player_y):
    """ group of memory patterns for goalkeeper """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # player is a goalkeeper have the ball
        if (obs["ball_owned_player"] == obs["active"] and
                obs["ball_owned_team"] == 0 and
                obs["ball_owned_player"] == 0):
            return True
        return False
        
    def get_memory_patterns(obs, player_x, player_y):
        """ get list of memory patterns """
        memory_patterns = [
            far_from_goal_shot,
            idle
        ]
        return memory_patterns
        
    return {"environment_fits": environment_fits, "get_memory_patterns": get_memory_patterns}

def offence_memory_patterns(obs, player_x, player_y):
    """ group of memory patterns for environments in which player's team has the ball """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # player have the ball
        if obs["ball_owned_team"] == 0:
            return True
        return False
        
    def get_memory_patterns(obs, player_x, player_y):
        """ get list of memory patterns """
        memory_patterns = [
            continue_after_gain_poss,
            far_from_goal_goalie_has_ball,
            far_from_goal_shot,
            far_from_goal_high_pass,
            bad_angle_short_pass,
            close_to_goalkeeper_shot,
            go_through_opponents,
            idle
        ]
        return memory_patterns
        
    return {"environment_fits": environment_fits, "get_memory_patterns": get_memory_patterns}

def other_memory_patterns(obs, player_x, player_y):
    """ group of memory patterns for all other environments """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        return True
        
    def get_memory_patterns(obs, player_x, player_y):
        """ get list of memory patterns """
        memory_patterns = [
            idle
        ]
        return memory_patterns
        
    return {"environment_fits": environment_fits, "get_memory_patterns": get_memory_patterns}

def special_game_modes_memory_patterns(obs, player_x, player_y):
    """ group of memory patterns for special game mode environments """
    def environment_fits(obs, player_x, player_y):
        """ environment fits constraints """
        # if game mode is not normal
        if obs['game_mode'] != GameMode.Normal:
            return True
        if (obs['active'] != 6 and obs['ball'][0] == 0 and obs['ball'][1] == 0 and
            obs['ball_direction'][0] == 0 and obs['ball_direction'][1] == 0 and
            obs['ball_rotation'][0] == 0 and obs['ball_rotation'][1] == 0):
            return True
        return False
        
    def get_memory_patterns(obs, player_x, player_y):
        """ get list of memory patterns """
        memory_patterns = [
            corner,
            free_kick,
            goal_kick,
            kick_off,
            penalty,
            throw_in,
            idle
        ]
        return memory_patterns
        
    return {"environment_fits": environment_fits, "get_memory_patterns": get_memory_patterns}

# Global Variables

# list of groups of memory patterns
groups_of_memory_patterns = [
    special_game_modes_memory_patterns,
    goalkeeper_memory_patterns,
    offence_memory_patterns,
    regular_defence_memory_patterns,
    defence_memory_patterns,
    other_memory_patterns
]

# dictionary of sticky actions
sticky_actions = {
    "left": Action.Left,
    "top_left": Action.TopLeft,
    "top": Action.Top,
    "top_right": Action.TopRight,
    "right": Action.Right,
    "bottom_right": Action.BottomRight,
    "bottom": Action.Bottom,
    "bottom_left": Action.BottomLeft,
    "sprint": Action.Sprint,
    "dribble": Action.Dribble
}

# @human_readable_agent wrapper modifies raw observations 
# provided by the environment:
# https://github.com/google-research/football/blob/master/gfootball/doc/observation.md#raw-observations
# into a form easier to work with by humans.
# Following modifications are applied:
# - Action, PlayerRole and GameMode enums are introduced.
# - 'sticky_actions' are turned into a set of active actions (Action enum)
#    see usage example below.
# - 'game_mode' is turned into GameMode enum.
# - 'designated' field is removed, as it always equals to 'active'
#    when a single player is controlled on the team.
# - 'left_team_roles'/'right_team_roles' are turned into PlayerRole enums.
# - Action enum is to be returned by the agent function.

memory = []
notRunning = [False,0]
did_init=False
state = None
steps_left = 3001

class My_State:
    def __init__(self):
        self.prev_active = None
        self.prev_ball_owned_team = None
        self.prev_ball_owned_player = None
        self.pass_cmd_issued = False
        self.quick_pass_cmd_issued = False
        self.quick_pass_cmd = None
        self.chase = False
        self.chase_cnt = 0
        self.current_score_left = 0
        self.current_score_right = 0
        self.debug_passing = False
        self.debug_defense = False
        self.debug_offense = False
        self.debug_special = False
        self.debug_distance = False

def init(obs):
    global state
    state = My_State()

# Execute a sequence of actions
def do_actions(sequence):
    global memory
    memory = sequence[1:]
    return sequence[0]

my_dist_own = [ [ 0 for y in range(11) ] for x in range(11) ]
my_angle_own = [ [ 5 for y in range(11) ] for x in range(11) ]
my_angle_goal = [ [ 5 for y in range(1) ] for x in range(11) ]
my_dist_opp = {}
my_angle_opp = {}
def_dist = [ [ 0 for y in range(11) ] for x in range(11) ]
def_angle = [ [ 0 for y in range(11) ] for x in range(11) ]
defender_near = [ [ 0 for y in range(1) ] for x in range(11) ]
defender_close = [ [ 0 for y in range(1) ] for x in range(11) ]
avg_dist_opp = [ [ 0 for y in range(1) ] for x in range(11) ]
def_block_pass_angle = [ [ None for y in range(11) ] for x in range(11) ]
defender_block_pass = [ [ 0 for y in range(11) ] for x in range(11) ]
defender_block_goal = [ [ 0 for y in range(1) ] for x in range(11) ]
def_block_goal_angle = {}
potential = {}

def block_angle_by_dist(dist):
    if dist < 0.1: return 12
    elif dist < 0.2: return 13
    elif dist < 0.3: return 15
    elif dist < 0.4: return 20
    elif dist < 0.5: return 25
    return 30

def angle_covered(goal_angle,opp_angle,block_angle):
    delta = abs(goal_angle-opp_angle)
    if  delta > 180:
        delta = 360-delta
    if delta <= block_angle:
        return True
    return False

def players_assess(obs):
    active = obs['active']
    left_team = obs['left_team']
    right_team = obs['right_team']
    for ix in range(11):
        if (state.debug_distance and ix == active):
            print(f'Step {3001-steps_left} P{ix} {left_team[ix][0]:.2f},{left_team[ix][1]:.2f}')
        # Calculate distance to our teammates
        for jx in range(ix+1,11):
            dist = get_distance(left_team[ix][0], left_team[ix][1], left_team[jx][0], left_team[jx][1])
            my_dist_own[ix][jx] = my_dist_own[jx][ix] = dist
        # Calculate angle to our teammates
        for jx in range(11):
            if ix == jx: continue
            angle = get_angle(left_team[ix], left_team[jx])
            my_angle_own[ix][jx] = angle
            if (state.debug_distance and ix == active and my_dist_own[ix][jx] < 0.2):
                print(f' P{jx} D:{my_dist_own[ix][jx]:.2f} A:{angle:.2f}')
        # Calculate distance and angle to our opponents
        opp_dist = {}
        opp_angle = {}
        for jx in range(11):
            dist = get_distance(left_team[ix][0], left_team[ix][1], right_team[jx][0], right_team[jx][1])
            opp_dist[jx] = dist
            def_dist[jx][ix] = dist
            angle = get_angle(left_team[ix], right_team[jx])
            opp_angle[jx] = angle
            def_angle[jx][ix] = 180+angle if angle < 0 else angle-180
            if state.debug_distance and ix == active and dist < 0.2:
                print(f' OP{jx} {right_team[jx][0]:.2f},{right_team[jx][1]:.2f} D:{dist:.2f} A:{angle:.2f}')
        my_dist_opp[ix] = opp_dist
        my_angle_opp[ix] = opp_angle
        my_angle_goal[ix] = get_angle(left_team[ix], (1,0))
        
    # Calculate current defender near and close.
    for ix in range(11):
        dist, amount = get_average_distance_to_opponents(obs,left_team[ix][0],left_team[ix][1])
        avg_dist_opp[ix] = dist/1000
        defender_near[ix] = defender_close[ix] = 0
        goal_angle = my_angle_goal[ix]
        def_block_goal = {}
        def_block_pass = {}
        for jx in range(11):
            if obs['right_team_active'][jx] is False:
                continue
            if my_dist_opp[ix][jx] <= 0.015*5:
                defender_near[ix] += 1
                if my_dist_opp[ix][jx] <= 0.015*2.5:
                    defender_close[ix] += 1
            opp_angle = my_angle_opp[active][jx]
            block_angle = block_angle_by_dist(my_dist_opp[active][jx])
            if angle_covered(goal_angle,opp_angle,block_angle):
                def_block_goal[jx] = opp_angle
            if active != ix and my_dist_own[active][ix] >= my_dist_opp[active][jx]:
                pass_angle = my_angle_own[active][ix]
                if angle_covered(pass_angle,opp_angle,block_angle):
                    def_block_pass[jx] = opp_angle
        def_block_goal_angle[ix] = def_block_goal
        defender_block_goal[ix] = len(def_block_goal_angle[ix])
        def_block_pass_angle[active][ix] = def_block_pass
        defender_block_pass[active][ix] = len(def_block_pass_angle[active][ix])
        if state.debug_distance and (defender_near[ix] == 0 or defender_close[ix] == 0 or defender_block_pass[active][ix] == 0 or ix == active):
            print(f' D:A:N:C:B {ix} {my_dist_own[active][ix]:.3f}:{avg_dist_opp[ix]:.3f}:{defender_near[ix]}:{defender_close[ix]}:{defender_block_pass[active][ix]}')

    for ix in range(11):
        # Calculate our player's potential
        potential[ix] = left_team[ix][0]-abs(left_team[ix][1])+avg_dist_opp[ix]-(my_dist_own[active][ix]/2.5)

@human_readable_agent
def agent(obs):
    global state
    global did_init
    global steps_left
    global memory
    
    if not did_init:
        init(obs)
        did_init=True

    # Execute memorized actions
    if (memory and state.prev_active == obs["active"] and
        state.prev_ball_owned_team == obs["ball_owned_team"] and
        state.prev_ball_owned_player == obs["ball_owned_player"]):
        return memory.pop(0)
    else:
        memory.clear()
    steps_left = obs['steps_left']
    active = obs['active']
    players_assess(obs)
    """ Ole ole ole ole """
    # dictionary for Memory Patterns data
    obs["memory_patterns"] = {}
    # We always control left team (observations and actions
    # are mirrored appropriately by the environment).
    controlled_player_pos = obs["left_team"][obs["active"]]
    if state.current_score_left != obs["score"][0] or state.current_score_right != obs["score"][1]:
        if state.debug_offense or state.debug_defense or state.debug_special or state.debug_passing:
            print(f'!!!!! Step {3001-steps_left} Score Update !!!!!! L:{obs["score"][0]} R:{obs["score"][1]}'
                  f' (previous {state.current_score_left}:{state.current_score_right})')
        state.current_score_left = obs["score"][0]
        state.current_score_right = obs["score"][1]
    # get action of appropriate pattern in agent's memory
    action = get_action_of_agent(obs, controlled_player_pos[0], controlled_player_pos[1])
    # return action
    state.prev_ball_owned_team = obs["ball_owned_team"]
    state.prev_ball_owned_player = obs["ball_owned_player"]
    state.prev_active = obs["active"]
    return action

In [ ]:
from kaggle_environments import make
deterministic = True
env = make("football", debug=True, configuration={"save_video": True, "scenario_name": "11_vs_11_deterministic" if deterministic else "11_vs_11_kaggle", "running_in_notebook": True, "episodeSteps": 3000})
output = env.run(["/kaggle/working/submission.py", "builtin_ai"])[-1]
print('Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))
env.render(mode="human", width=800, height=600)

In [ ]:
from kaggle_environments import make
def playMatch(leftAgent = "builtin_ai", rightAgent = "builtin_ai", deterministic = False):
    env = make("football", debug=True, configuration={"save_video": True, "scenario_name": "11_vs_11_deterministic" if deterministic else "11_vs_11_kaggle", "running_in_notebook": True, "episodeSteps": 3000})
    output = env.run([leftAgent, rightAgent])[-1]
    print('Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
    print('Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))
    env.render(mode="human", width=800, height=600)
    return env, output

from pandas import DataFrame

def analyse_match(steps):
    obs = []
    for step in steps:
        val = step[0]['observation']['players_raw'][0] # left player
        val['action'] = step[0]['action'][0] if step[0]['action'] else -1
        obs.append(val)
    df = DataFrame(obs)
    #print(f'{dir(obs)} {obs[99]}')
    lgoal = obs[-1]['score'][0]
    rgoal = obs[-1]['score'][1]
    lwin = rwin = False
    if lgoal > rgoal:
        lwin = True
    elif lgoal < rgoal:
        rwin = True
    pos = [-1,-1,-1]
    pos = (df['ball_owned_team'].value_counts()/sum(df['ball_owned_team'].value_counts())*100).rename({-1:'None',0:'Left',1:'Right'})
    print(f'{pos[0]} {pos[1]} {pos[2]}')
    return lwin, rwin, lgoal, rgoal, pos[1], pos[2]

def printStats(steps):
    obs = []
    for step in steps:
        val = step[0]['observation']['players_raw'][0] # left player
        val['action'] = step[0]['action'][0] if step[0]['action'] else -1
        obs.append(val)
    df = DataFrame(obs)
    print(f'{dir(obs)} {obs[99]}')
    report = {}
    report['Goal Scored'] = obs[-1]['score']
    report['Yellow Cards'] = [sum(obs[-1]['left_team_yellow_card']),sum(obs[-1]['right_team_yellow_card'])]
    report['Red Cards'] = [11-sum(obs[-1]['left_team_active']),11-sum(obs[-1]['right_team_active'])]
    reportDF = DataFrame.from_records(report).T
    reportDF.columns = ['Left','Right']

    ballOwn = (df['ball_owned_team'].value_counts()/sum(df['ball_owned_team'].value_counts())*100).rename({-1:'None',0:'Left',1:'Right'})
    gameMode = (df['game_mode'].value_counts()/sum(df['game_mode'].value_counts())*100).rename({0:'Normal',1:'KickOff',2:'GoalKick',3:'FreeKick',4:'Corner',5:'ThrowIn',6:'Penalty'})
    ballPos = DataFrame.from_records(df['ball'])
    ballPos.columns = ['x','y','z']

    print('-----------------------------\n   Stats                \n-----------------------------')
    print(reportDF)
    print('-----------------------------\n   Ball Possession (%)  \n-----------------------------')
    print(ballOwn.to_string())
    print('-----------------------------\n   Game Mode (%)        \n-----------------------------')
    print(gameMode.to_string())
    print('-----------------------------\n   Average Ball Position\n-----------------------------')
    print(ballPos.mean().to_string())

In [ ]:
leftwin = rightwin = draw = 0
leftpos = rightpos = 0
leftgoal = rightgoal = 0
totalgame = 5
lgoal = [ [ 0 for y in range(1) ] for x in range(totalgame) ]
rgoal = [ [ 0 for y in range(1) ] for x in range(totalgame) ]

for game in range(totalgame):
    match, output = playMatch("/kaggle/working/submission.py", deterministic=False)
    lwin, rwin, lgoal[game], rgoal[game], lpos, rpos = analyse_match(match.steps)
    if lwin:
        leftwin += 1
    elif rwin:
        rightwin += 1
    else:
        draw += 1
    leftpos += lpos
    rightpos += rpos
    leftgoal += lgoal[game]
    rightgoal += rgoal[game]
    print(f'Game {game+1}, Left:{lgoal[game]} Right:{rgoal[game]} ({lpos}):({rpos})')

print(f'V20_4 vs AI Left wins {leftwin}, loses {rightwin}, draw {draw}, ({leftgoal}:{rightgoal}) {(leftgoal*100/(rightgoal if rightgoal > 0 else 1)):.2f}% (Lpos:{leftpos/totalgame} Rpos:{rightpos/totalgame})')

In [ ]:
# Replay analysis of gfootball game from json file
from kaggle_environments.envs.football.helpers import *
import json
import math

class My_State:
    def __init__(self):
        self.ball_owner = None
        self.intended_passer = None
        self.intended_passee = None
        self.completed_pass = 0
        self.offside_value = 0
        self.debug_distance = False
        self.debug_distance_future = False
        self.debug_passing = False
        self.debug_passing_detail = False
        self.debug_defense = False
        self.debug_offense = True
        self.debug_future = False
        self.debug_passing_super_detail = False

def init():
    global state
    state = My_State()
    
def calculate_potential(filename, start, stop):
    global potential
    init()

    def_dist = [ [ 0 for y in range(11) ] for x in range(11) ]
    def_angle = [ [ 0 for y in range(11) ] for x in range(11) ]
    def_angle_goal = [ [ 0 for y in range(1) ] for x in range(11) ]
    my_dist_own = [ [ 0 for y in range(11) ] for x in range(11) ]
    def_block_pass_angle = [ [ None for y in range(11) ] for x in range(11) ]
    defender_block_pass = [ [ 0 for y in range(11) ] for x in range(11) ]
    potential = [ [ 5 for y in range(1) ] for x in range(11) ]
    defender_near = [ [ 5 for y in range(1) ] for x in range(11) ]
    defender_close = [ [ 5 for y in range(1) ] for x in range(11) ]
    avg_dist_opp = [ [ 0 for y in range(1) ] for x in range(11) ]
    defender_block_goal = [ [ 5 for y in range(1) ] for x in range(11) ]
    goal_dist_own = [ [ 5 for y in range(1) ] for x in range(11) ]
    goal_dist_opp = [ [ 5 for y in range(1) ] for x in range(11) ]
    my_open_goal_angle = [ [ 5 for y in range(1) ] for x in range(11) ]
    ball_dist = [ [ 5 for y in range(1) ] for x in range(11) ]
    my_angle_goal = [ [ 5 for y in range(1) ] for x in range(11) ]
    my_angle_own = [ [ 5 for y in range(11) ] for x in range(11) ]
    my_future_dist_own = [ [ 0 for y in range(11) ] for x in range(11) ]
    future_def_block_pass_angle = [ [ None for y in range(11) ] for x in range(11) ]
    future_defender_block_pass = [ [ 0 for y in range(11) ] for x in range(11) ]
    future_left_team = [ [ 5 for y in range(1) ] for x in range(11) ]
    future_right_team = [ [ 5 for y in range(1) ] for x in range(11) ]
    future_defender_near = [ [ 5 for y in range(1) ] for x in range(11) ]
    future_defender_close = [ [ 5 for y in range(1) ] for x in range(11) ]
    future_defender_block_goal = [ [ 5 for y in range(1) ] for x in range(11) ]
    goal_future_dist_own = [ [ 5 for y in range(1) ] for x in range(11) ]
    goal_future_dist_opp = [ [ 5 for y in range(1) ] for x in range(11) ]
    my_future_open_goal_angle = [ [ 5 for y in range(1) ] for x in range(11) ]
    ball_future_dist = [ [ 5 for y in range(1) ] for x in range(11) ]
    my_future_angle_goal = [ [ 5 for y in range(1) ] for x in range(11) ]
    my_future_angle_own = [ [ 5 for y in range(11) ] for x in range(11) ]
    potential = {}
    my_dist_opp = {}
    my_angle_opp = {}
    def_block_goal_angle = {}
    my_future_dist_opp = {}
    my_future_angle_opp = {}
    future_def_block_goal_angle = {}
    def get_distance(x1, y1, x2, y2):
        """ get two-dimensional Euclidean distance, considering y size of the field """
        return math.sqrt((x1 - x2) ** 2 + (y1 * 2.38 - y2 * 2.38) ** 2)
    def get_average_distance_to_opponents(obs, player_x, player_y):
        """ get average distance to closest opponents """
        distances_sum = 0
        distances_amount = 0
        for i in range(1, len(obs["right_team"])):
            # if opponent is ahead of player
            if obs["right_team"][i][0] > (player_x - 0.015):
                distance_to_opponent = get_distance(player_x, player_y, obs["right_team"][i][0], obs["right_team"][i][1])
                if distance_to_opponent < 0.1:
                    distances_sum += distance_to_opponent
                    distances_amount += 1
        # if there is no opponents close around
        if distances_amount == 0:
            return 100, distances_amount
        return distances_sum*10000 // (distances_amount*10), distances_amount

    # Evaluate euclidian distance between two objects
    def uncalibrated_get_distance(pos1,pos2):
        return (get_distance(pos1[0],pos1[1],pos2[0],pos2[1]))    # Evaluate euclidian distance between two objects
    # Evaluate angle between two objects
    def get_angle(pos1,pos2):
        return math.degrees(math.atan2(pos2[1]-pos1[1], pos2[0]-pos1[0]))
    def get_direction(pass_angle):
        action, angle = actual_pass_direction(pass_angle)
        return action, angle
    def block_angle_by_dist(dist):
        if dist < 0.1: return 7
        elif dist < 0.2: return 10
        elif dist < 0.3: return 15
        elif dist < 0.4: return 20
        return 25
    def angle_adjust(ang):
        if ang > 180: return ang-360
        elif ang < -180: return 360+ang
        else: return ang

    def angle_covered(goal_angle,opp_angle,block_angle):
        delta = abs(goal_angle-opp_angle)
        if  delta > 180:
            delta = 360-delta
        if delta <= block_angle:
            return True
        return False
    def pos_after_move(pos,dir,mulp):
        # high dir means high speed so we use high multiplier
        # because it takes a long time to change to do new thing.
        # low dir, low multiplier because there's low delay to change.
        return (pos[0]+dir[0]*mulp,pos[1]+dir[1]*mulp)

    def players_assess(obs):
        active = obs['active']
        left_team = obs['left_team']
        right_team = obs['right_team']
        for ix in range(11):
            if (state.debug_distance and ix == active):
                print(f'Step {3001-steps_left} P{ix} {left_team[ix][0]:.2f},{left_team[ix][1]:.2f}')
            ball_dist[ix] = get_distance(left_team[ix][0], left_team[ix][1], obs["ball"][0], obs["ball"][1])
            # Calculate distance to our teammates
            for jx in range(ix+1,11):
                dist = get_distance(left_team[ix][0], left_team[ix][1], left_team[jx][0], left_team[jx][1])
                my_dist_own[ix][jx] = my_dist_own[jx][ix] = dist
            # Calculate angle to our teammates
            for jx in range(11):
                if ix == jx: continue
                angle = get_angle(left_team[ix], left_team[jx])
                my_angle_own[ix][jx] = angle
                if (state.debug_distance and ix == active and my_dist_own[ix][jx] < 0.2):
                    print(f' P{jx} D:{my_dist_own[ix][jx]:.2f} A:{angle:.2f}')
            # Calculate distance and angle to our opponents
            opp_dist = {}
            opp_angle = {}
            for jx in range(11):
                dist = get_distance(left_team[ix][0], left_team[ix][1], right_team[jx][0], right_team[jx][1])
                opp_dist[jx] = dist
                def_dist[jx][ix] = dist
                angle = get_angle(left_team[ix], right_team[jx])
                opp_angle[jx] = angle
                def_angle[jx][ix] = 180+angle if angle < 0 else angle-180
                if state.debug_distance and ix == active and dist < 0.2:
                    print(f' OP{jx} {right_team[jx][0]:.2f},{right_team[jx][1]:.2f} D:{dist:.2f} A:{angle:.2f}')
            my_dist_opp[ix] = opp_dist
            my_angle_opp[ix] = opp_angle
            my_angle_goal[ix] = get_angle(left_team[ix], (1,0))

        # Calculate current defender near and close.
        for ix in range(11):
            dist, amount = get_average_distance_to_opponents(obs,left_team[ix][0],left_team[ix][1])
            avg_dist_opp[ix] = dist/1000
            defender_near[ix] = defender_close[ix] = 0
            goal_angle = my_angle_goal[ix]
            def_block_goal = {}
            def_block_pass = {}
            for jx in range(11):
                if obs['right_team_active'][jx] is False:
                    continue
                if my_dist_opp[ix][jx] <= 0.015*5:
                    defender_near[ix] += 1
                    if my_dist_opp[ix][jx] <= 0.015*2.5:
                        defender_close[ix] += 1
                opp_angle = my_angle_opp[active][jx]
                block_angle = block_angle_by_dist(my_dist_opp[active][jx])
                if angle_covered(goal_angle,opp_angle,block_angle):
                    def_block_goal[jx] = opp_angle
                if active != ix and my_dist_own[active][ix] >= my_dist_opp[active][jx]:
                    pass_angle = my_angle_own[active][ix]
                    if angle_covered(pass_angle,opp_angle,block_angle):
                        def_block_pass[jx] = opp_angle
            def_block_goal_angle[ix] = def_block_goal
            defender_block_goal[ix] = len(def_block_goal_angle[ix])
            def_block_pass_angle[active][ix] = def_block_pass
            defender_block_pass[active][ix] = len(def_block_pass_angle[active][ix])
            if state.debug_distance and (defender_near[ix] == 0 or defender_close[ix] == 0 or defender_block_pass[active][ix] == 0 or ix == active):
                print(f' D:A:N:C:B {ix} {my_dist_own[active][ix]:.3f}:{avg_dist_opp[ix]:.3f}:{defender_near[ix]}:{defender_close[ix]}:{defender_block_pass[active][ix]}')

        for ix in range(11):
            # Calculate our player's potential
            potential[ix] = left_team[ix][0]-abs(left_team[ix][1])+avg_dist_opp[ix]-(my_dist_own[active][ix]/2.5)

    # Estimate landing position
    def ball_landing_pos(obs):
        start_height = obs['ball'][2]
        end_height = 0.2
        start_speed = obs['ball_direction'][2]
        gravity = 0.1
        time = (start_speed**2/gravity**2 - 2/gravity*(end_height-start_height))**0.5 + start_speed/gravity
        ex = obs['ball'][0]+obs['ball_direction'][0]*time
        ey = obs['ball'][1]+obs['ball_direction'][1]*time
        if state.debug_offense:
            print(f'ball_landing_pos: Time {time:.2f} loc {ex:.3f},{ey:.3f}')
        return (ex,ey), time

    def ball_intercept_pos(obs):
        active = obs['active']
        va = obs['left_team_direction'][active]
        loca = obs['left_team'][active]
        vb = obs['ball_direction']
        locb = obs['ball']
        dvx = vb[0]-va[0]
        dvy = vb[1]-va[1]
        tx = ty = 0
        if dvx != 0:
            tx = (loca[0]-locb[0])/dvx
        if dvy != 0:
            ty = (loca[1]-locb[1])/dvy
        if dvx == 0 and dvy == 0:
            return (locb[0],locb[1]), 10
        time = min(abs(tx),abs(ty))
        ex = locb[0]+vb[0]*time
        ey = locb[1]+vb[1]*time
        if state.debug_offense:
            print(f'Ball_intercept_pos: Time {time:.2f} x:{tx:.2f} t:{ty:.2f} loc {ex:.3f},{ey:.3f}')
        return (ex,ey), time

    # Estimate ball movement to intercept
    def estimate_ball_pos(obs):
        if obs['ball'][2] > 0.3:
            loc, time = ball_landing_pos(obs)
        else:
            loc, time = ball_intercept_pos(obs)
        return loc, time

    def find_evade_dir(obs, player_x, player_y):
        biggest_distance = 0
        left, left_opponents_amount = get_average_distance_to_opponents(obs, player_x - 0.01, player_y)
        right, right_opponents_amount = get_average_distance_to_opponents(obs, player_x + 0.01, player_y)
        top_right, tr_opponents_amount = get_average_distance_to_opponents(obs, player_x + 0.01, player_y - 0.01)
        top_left, tl_opponents_amount = get_average_distance_to_opponents(obs, player_x - 0.01, player_y - 0.01)
        top, t_opponents_amount = get_average_distance_to_opponents(obs, player_x, player_y - 0.01)
        if player_y <= -0.32:
            top_right = top_left = top = 0
        bottom_right, br_opponents_amount = get_average_distance_to_opponents(obs, player_x + 0.01, player_y + 0.01)
        bottom_left, bl_opponents_amount = get_average_distance_to_opponents(obs, player_x - 0.01, player_y + 0.01)
        bottom, b_opponents_amount = get_average_distance_to_opponents(obs, player_x, player_y + 0.01)
        if player_y >= 0.32:
            bottom_right = bottom_left = bottom = 0
        if player_x <= -0.7 and player_y >= -0.2:
            bottom_left = left = 0
        if player_x <= -0.7 and player_y <= 0.2:
            top_left = left = 0
        if player_x <= -0.85:
            top_left = bottom_left = left = 0
        if player_x > 0.93:
            top_left = top_right = right = 0
        biggest_distance = max(left, right, top_right, top_left, top, bottom_right, bottom_left, bottom)
        action = Action.Right
        if (biggest_distance == right or right == 100):
            # Right is best or clear and already in the center or wing
            action = Action.Right
        elif (biggest_distance == top_right or top_right == 100):
            action = Action.TopRight
        elif (biggest_distance == bottom_right or bottom_right == 100):
            action = Action.BottomRight
        elif (biggest_distance == top or top == 100):
            action = Action.Top
        elif (biggest_distance == bottom or bottom == 100):
            action = Action.Bottom
        elif (biggest_distance == top_left or top_left == 100):
            action = Action.TopLeft
        elif (biggest_distance == bottom_left or bottom_left == 100):
            action = Action.BottomLeft
        elif (biggest_distance == left or left == 100):
            action = Action.Left
        print(f'Step {ix-1} BL:L:TL {bottom_left}:{left}:{top_left} T:B {top}:{bottom} TR:R:BR {top_right}:{right}:{bottom_right}')
        return action

    with open(filename, "r") as read_file:
        dev = json.load(read_file)
        for ix in range(start,stop+1):
            obs = dev["steps"][ix][0]["observation"]['players_raw'][0]
            oppobs = dev["steps"][ix][1]["observation"]['players_raw'][0]
            left_action = dev["steps"][ix][0]["action"][0]
            right_action = dev["steps"][ix][1]["action"][0]
            active = obs['active']
            oppactive = oppobs['active']
            ball_owned_team = obs['ball_owned_team']
            controlled_player_pos = obs['left_team'][active]
            opp_player_pos = obs['right_team'][oppactive]
            dir = obs['left_team_direction'][active]
            oppdir = obs['right_team_direction'][oppactive]
            goaliepos = obs['right_team'][0]
            goaliedir = obs['right_team_direction'][0]
            ball_dir = obs['ball_direction']
            ball = obs['ball']
            next_ball = obs['ball']+ball_dir
            players_assess(obs)
            #print(f'obs[{ix}] {obs}')
            xstep = ystep = 0
            dx = opp_player_pos[0]-controlled_player_pos[0]
            dy = opp_player_pos[1]-controlled_player_pos[1]
            if oppdir[0]-dir[0] != 0:
                xstep = (opp_player_pos[0]-controlled_player_pos[0])/(oppdir[0]-dir[0])
            if oppdir[1]-dir[1] != 0:
                ystep = (opp_player_pos[1]-controlled_player_pos[1])/(oppdir[1]-dir[1])
            ball_land_pos, time = estimate_ball_pos(obs)
            evade_dir = find_evade_dir(obs, controlled_player_pos[0], controlled_player_pos[1])
            print(f'Step {ix-1} Bown {ball_owned_team}'
                  f'\n We {active} ({controlled_player_pos[0]:.4f},{controlled_player_pos[1]:.4f})'
                  f' dir:({dir[0]:.4f},{dir[1]:.4f}) C:N {defender_close[active]}:{defender_near[active]} Evade:{evade_dir}'
                  f'\n Op {oppactive} ({opp_player_pos[0]:.4f},{opp_player_pos[1]:.4f})'
                  f' dir:({oppdir[0]:.4f},{oppdir[1]:.4f})'
                  f'\n B ({ball[0]:.4f},{ball[1]:.4f},{ball[2]:.4f})'
                  f' dir:({ball_dir[0]:.4f},{ball_dir[1]:.4f},{ball_dir[2]:.4f})'
                  f'\n Exp. B land ({ball_land_pos[0]:.4f},{ball_land_pos[1]:.4f} time {time:.2f})'
                  f'\n OpG ({goaliepos[0]:.4f},{goaliepos[1]:.4f})'
                  f' dir:({goaliedir[0]:.4f},{goaliedir[1]:.4f})')
            print(f'\n ATV:B:{ball_dist[active]:.4f} ATV:R:{my_dist_opp[active][oppactive]:.4f}'
                  f' ATV:G:{my_dist_opp[active][0]:.4f} xs:{xstep:.2f} ys:{ystep:.2f} dx:{dx:.3f} dy:{dy:.3f}')
            print(f'Step {ix-1} We action {left_action} Op action {right_action}\n')
            #num, tgt_loc, tgt_dir = find_threat()
            #print(f'  Threat {num} Loc:{tgt_loc[0]:.2f},{tgt_loc[1]:.2f} Dir:{tgt_dir[0]:.2f},{tgt_dir[1]:.2f}')
            '''
            if ball_owned_team == 0:
                num, dir = teammate_more_open(active)
                #print(f'Step {ix-1} pass {num} dir {dir}')
                if num == 0:
                    action = dribble(obs)
                    # print(f'  dribble {action}')
                sort_potential = sorted(potential.items(), key=lambda item: item[1], reverse=True)
                #print(f'sort pot: {sort_potential}')
                #print(f'pot: {potential}')
                sticky = obs['sticky_actions']
                #print(f'sticky: {sticky} {Action.Right}')
            elif ball_owned_team == 1:
                num,tgt,tgt_dir = find_threat()
                opp = (opp_player_pos[0]+oppdir[0],opp_player_pos[1]+oppdir[1])
                if (state.debug_defense):
                    print(f'Step {ix-1} {active} {oppactive}, opp:{opp_player_pos[0]:.3f},{opp_player_pos[1]:.3f} dir:{oppdir[0]:.3f},{oppdir[1]:.3f} result:{opp[0]:.3f},{opp[1]:.3f}') 
                mark(tgt)
            '''

calculate_potential("../input/gameplay/5925837_Hui_vecxoz.json",1835,1845)